## NOMS Paper NANOG (5.2)

In [ ]:
using CSV
using DataFrames
using Distributions
using Glob
using HMMBase
using JSON
using ParsimoniousMonitoring: LogEntry
using PyPlot
using Statistics
using ThesisTools

In [ ]:
parse_log(d::Dict) = LogEntry(map(x -> d[string(x)], fieldnames(LogEntry))...)

function load_results(filename)
    obj = JSON.parsefile(filename)
    Dict(k => map(parse_log, v) for (k, v) in obj)
end

# TODO: Move to dedicated file
read_ts(filename) = CSV.read(filename, header = ["timestep", "rtt"])

function load_scenario(path; return_names = false)
    files = map(x -> splitext(x)[1], glob("*.csv", path))
    models = []
    series = []
    names = []
    for file in files
        push!(models, read_model("$file.json"))
        push!(series, read_ts("$file.csv"))
        push!(names, basename(file))
    end
    data = hcat(map(x -> x.rtt, series)...)
    name = splitpath(path)[end]
    if return_names
        return name, names, models, data
    end
    name, models, data
end

function read_model(filename)
    obj = JSON.parsefile(filename)
    A = permutedims(hcat(obj["transmat"]...))
    B = map(obj["states"]) do (_, d)
        components = map(zip(d["means"], d["variances"])) do (μ, σ2)
            Normal(μ, sqrt(σ2))
        end
        MixtureModel(components, [d["weights"]...])
    end
    HMM(A, B)
end

### 5.2 Validation against real latency data

In [ ]:
scenarios = [
    "../data/noms_scenarios/Haifa_Santiago/",
    "../data/noms_scenarios/Paris_Santiago/",
    "../data/noms_scenarios/Paris_Tokyo/",
    "../data/noms_scenarios/Singapore_HongKong/"
];

In [ ]:
τmax = 100
for file in scenarios
    models = load_scenario(file)[2]
    @show file
    @show prod(m -> size(m, 1) * τmax, models)
end

In [ ]:
files = [
    "../results/Haifa_Santiago.json",
    "../results/Paris_Santiago.json",
    "../results/Paris_Tokyo.json",
    "../results/Singapore_HongKong.json"
]

function min_delay_gap(log)
    delay = sum(skipmissing(map(x -> isnothing(x.delay) ? missing : x.delay, log)))
    delay_opt = sum(skipmissing(map(x -> isnothing(x.delay_opt) ? missing : x.delay_opt, log)))
    (delay - delay_opt) / delay_opt
end

metrics = Dict(
    "processing_time" => log -> mean(x -> x.time, log),
    "avg_measures" => log -> mean(x -> sum(x.a), log),
    "min_delay_reached" => log -> mean(x -> x.delay == x.delay_opt, log) * 100,
    "min_delay_gap" => log -> min_delay_gap(log) * 100
);

In [ ]:
df = DataFrame([String[], String[], String[], Real[]], [:scenario, :policy, :metric, :value])
for file in files
    scenario = splitext(basename(file))[1]
    results = load_results(file)
    for (policy, log) in results, (metric, f) in metrics
        push!(df, (scenario, policy, metric, f(log)))
    end
end
df = unstack(df, :metric, :value);

In [ ]:
df[df.scenario .== "Singapore_HongKong",:]

**Figure Singapore - HK**

In [ ]:
name, names, models, data = load_scenario("../data/noms_scenarios/Singapore_HongKong/", return_names = true)
results = load_results("../results/Singapore_HongKong.json")
names

In [ ]:
fig, ax = subplots()
ax.plot(coalesce.(data[:,1], NaN), label = "Direct")
ax.plot(coalesce.(data[:,2], NaN), label = "via Narita")
ax.plot(coalesce.(data[:,3], NaN), label = "via Tokyo")
ax.scatter(1:size(data, 1), map(x -> coalesce(x.delay, NaN), results["Heuristic"]), c = "black", s = 10)
ax.legend()

**Figure Haifa - Santiago**

In [ ]:
name, names, models, data = load_scenario("../data/noms_scenarios/Haifa_Santiago/", return_names = true)
results = load_results("../results/Haifa_Santiago.json")
names

In [ ]:
@pywith plt.style.context("classicthesis") begin
    fig, ax = subplots(figsize = (4.6, 4.6*0.45))
    ax.plot(coalesce.(data[:,1], NaN), label = "Direct")
    ax.plot(coalesce.(data[:,2], NaN), label = "via Boston")
    ax.plot(coalesce.(data[:,3], NaN), label = "via Curitiba")
    ax.plot(coalesce.(data[:,4], NaN), label = "via Dublin")
    ax.scatter(1:size(data, 1), map(x -> coalesce(x.delay, NaN), results["RH-3"]), c = "black", s = 10)
    ax.set(xlabel = "Temps", ylabel = "RTT (ms)")
    ax.legend(loc = "upper left", ncol = 2)
    fig.savefig("../figures/haifa_santiago_rh3.pdf")
    # save_thesis("haifa_santiago_rh3", clean = true, hwr = 0.35)
end

In [ ]:
fig, axs = subplots(figsize = (10, 9), nrows = 3, sharex = true)
axs[1].plot(data[:,1], label = "Déterministe")
axs[1].plot(data[:,2], label = "Stochastique")
axs[1].scatter(instants, data[instants,2], ec = colorblindmap[3], fc = "none", marker = "o", label = "Mesure")
axs[1].set(ylabel = L"$L(t)$", ylim = (4, 12))
axs[1].legend(loc = "upper left", ncol = 3)

axs[2].plot(predictor)
# Bug with axhline and tikzplotlib
# axs[2].axhline(xmin, c = "black", ls = "--", lw = 1.0, label = "xmin")
# axs[2].axhline(xmax, c = "black", ls = "--", lw = 1.0, label = "xmax")
axs[2].plot([0, size(data, 1)], [xmin, xmin], c = "black", ls = "--", lw = 1.0, label = "xmin, xmax")
axs[2].plot([0, size(data, 1)], [xmax, xmax], c = "black", ls = "--", lw = 1.0)
axs[2].set(ylabel = L"\gamma_{t-1,t}(1)", ylim = (0, 1.0))
axs[2].legend(loc = "upper left", ncol = 2)

axs[3].plot(cumsum(penalized_gain(mdp, logbook_never)), label = "Jamais")
axs[3].plot(cumsum(penalized_gain(mdp, logbook_always)), label = "Toujours")
axs[3].plot(cumsum(penalized_gain(mdp, logbook_greedy)), label = "Glouton")
axs[3].plot(cumsum(penalized_gain(mdp, logbook_mdp_99)), label = "MDP 0.99")
axs[3].set(xlabel = "Temps", ylabel = "Gain pénalisé cumulé", ylim = (-500, 6000))
axs[3].legend(loc = "upper left", ncol = 2)
axs[3].grid()

save_thesis("monitoring_1p_greedy_aio", clean = true, hwr = 0.4)